In [2]:
import pandas as pd
import numpy as np
import time
from datetime import datetime


In [ ]:
# Convert a list full of string to python datetime format ，用于处理导入matlab数据中的时间戳
def Str2times(indx):
    z = list()
    for v in indx:
        z.append(datetime.strptime(v,'%Y-%m-%d'))
    return z 

# Convert matlab date format to python date format.
def ConTimeDay(data1):
    date = data1['Date']
    out = []
    for v in date:
        date=(v-719529)*86400+3600*8
        date = time.localtime(date)
        date = time.strftime('%Y-%m-%d',date)
        out.append(date)
    return out
def getTradingDayDataFromMat(data1,strs,tradingDay,isUseTrade,cuts):
    col = data1['LocalID']
    ind = ConTimeDay(data1) 
    ind = Str2times(ind )
    df = data1[strs] # 日开盘价
    df = pd.DataFrame(df)   # 转化Open为df文件
    df.columns  = col  # 设置
    df.index = ind
    df = df.iloc[cuts:]
    if isUseTrade == 1:
        df = df[df.index.isin(tradingDay.index)]
    return df

In [2]:
def get_raw_return_data():
    import mat73  # matlab格式数据导入模块 
    data1 = mat73.loadmat("Px.mat")  
    # get raw data
    tradingDay = getTradingDayDataFromMat(data1,'Volume',0,0,0)
    tradingDay = tradingDay[tradingDay.sum(1) > 0]   # get data with only trading day

    isUseTrade = 1
    cut = 1200  # Because memory or CPU is not enough： cut the early year's data

    TotalRet = getTradingDayDataFromMat(data1,'TotalRet',tradingDay,isUseTrade ,cut)# 日收益率  **
    TotalRet.to_pickle('TotalRet.pkl')  # 保存数据

ERROR:root:ERROR: MATLAB type not supported: table, (uint32)
ERROR:root:ERROR: MATLAB type not supported: table, (uint32)
ERROR:root:ERROR: MATLAB type not supported: table, (uint32)
ERROR:root:ERROR: MATLAB type not supported: table, (uint32)


{'ADV90': array([[           nan,            nan,            nan, ...,
                    nan,            nan,            nan],
        [           nan,            nan,            nan, ...,
                    nan,            nan,            nan],
        [           nan,            nan,            nan, ...,
                    nan,            nan,            nan],
        ...,
        [1.60958101e+09, 2.33878475e+09, 1.27968630e+08, ...,
                    nan,            nan,            nan],
        [1.60419824e+09, 2.37097978e+09, 1.23698444e+08, ...,
                    nan,            nan,            nan],
        [1.58732649e+09, 2.38070035e+09, 1.19068119e+08, ...,
                    nan,            nan,            nan]]),
 'AMktCap': array([[           nan,            nan,            nan, ...,
                    nan,            nan,            nan],
        [           nan,            nan,            nan, ...,
                    nan,            nan,            nan],
     

In [2]:
total_return = pd.read_pickle('TotalRet.pkl')

In [7]:
total_return = total_return.loc[total_return.index >= '2020-01-01', :]
total_return = total_return.loc[total_return.index <= '2023-12-31', :]
total_return.to_pickle('total_return.pkl')
total_return.T.to_csv('total_return.csv')  # 保存数据

In [5]:
zz500 = pd.read_csv('zz500.csv', index_col=0)
zz500.sort_index(inplace=True)
zz500.index = pd.to_datetime(zz500.index)

In [6]:
z

DatetimeIndex(['2020-01-02', '2020-01-03', '2020-01-06', '2020-01-07',
               '2020-01-08', '2020-01-09', '2020-01-10', '2020-01-13',
               '2020-01-14', '2020-01-15',
               ...
               '2023-12-18', '2023-12-19', '2023-12-20', '2023-12-21',
               '2023-12-22', '2023-12-25', '2023-12-26', '2023-12-27',
               '2023-12-28', '2023-12-29'],
              dtype='datetime64[ns]', name='日期', length=970, freq=None)

In [16]:
zz500.index = pd.to_datetime(zz500.index)

In [25]:
# 将带逗号的字符串转换为浮点数
for column in ['收盘', '开盘', '高', '低']:
    zz500[column] = zz500[column].str.replace(',', '').astype(float)

# 将交易量的字符串转换为浮点数，假设单位为千
zz500['交易量'] = zz500['交易量'].str.replace('K', '').astype(float) * 1000

# 将涨跌幅从字符串转换为浮点数
zz500['涨跌幅'] = zz500['涨跌幅'].str.replace('%', '').astype(float) / 100

In [ ]:
zz500

,收盘,开盘,高,低,交易量,涨跌幅
日期,,,,,,
2020-01-10,5480.23,5511.67,5516.09,5455.25,113820.0,-0.0031
2020-01-13,5559.63,5478.32,5559.63,5447.73,118080.0,0.0145
2020-01-14,5542.33,5576.63,5583.72,5539.90,129810.0,-0.0031
2020-01-15,5530.03,5532.96,5545.10,5492.65,112430.0,-0.0022
2020-01-16,5527.67,5539.42,5559.02,5514.69,111900.0,-0.0004
...,...,...,...,...,...,...
2023-09-04,5829.78,5772.03,5829.78,5758.03,135650.0,0.0137
2023-09-05,5788.71,5819.97,5822.06,5779.37,106520.0,-0.0070
2023-09-06,5790.38,5771.85,5800.96,5748.65,90480.0,0.0003


In [7]:
zz500.to_csv('zz500.csv')